<a href="https://colab.research.google.com/github/Aniket-Mitra/Meta-N-Ball/blob/main/Meta_N_Ball_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
#from google.colab import files
os.mkdir('HPO_results')
os.mkdir('HPO_results/EmEL_dir')
!unzip 'HPO_all_Mar1.zip'
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import click as ck
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.framework import function
import re
import math
import matplotlib.pyplot as plt
import logging
from tensorflow.keras.layers import (
    Input,
)
from tensorflow.keras import optimizers
from tensorflow.keras import constraints
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from tensorflow.keras import backend as K
from scipy.stats import rankdata
from tensorflow.python.keras.utils.data_utils import Sequence

config = tf.compat.v1.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
#sess = tf.compat.v1.Session()
#K.set_session(sess)
#tf.keras.backend.set_session(session)
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

logging.basicConfig(level=logging.INFO)



def load_valid_data(valid_data_file, classes, instances,relations):
    data = []
    rel1 = f'SubClassOf'
    rel2=f'InstanceOf'
    #ccccx=list(classes.keys())
    with open(valid_data_file, 'r') as f:
        for line in f:
          it = line.strip().split()
          if len(it)==2:
            id1 = it[0]
            id2 = it[1]
            data.append((classes[id1], relations[rel1], classes[id2]))
          elif len(it)==3:
            id1 = it[0]
            id2 = it[1]
            id3 = it[2]
            data.append((instances[id1],relations[rel2],classes[id2],relations[rel1],classes[id3]))

    return data




def load_cls(train_data_file):
    train_subs=list()
    counter=0
    with open(train_data_file,'r') as f:
        for line in f:
            counter+=1
            it = line.strip().split()
            cls1 = it[0]
            cls2 = it[1]
            train_subs.append(cls1)
            train_subs.append(cls2)
    train_cls = list(set(train_subs))
    return train_cls,counter

In [ ]:
def load_data(filename):
    classes = {}
    import pickle
    relations = {}
    instances={}
    data = {'nf1': [], 'nf1_ins':[],'nf2': [], 'nf3': [], 'nf4': [], 'disjoint': [],'nf_inclusion':[],'nf_chain':[]}
    with open('rank_dict_hv_hp2.pkl', 'rb') as f:
            rank_dict = pickle.load(f)
    with open(filename) as f:
        #with open('rank_dict_hv2.pkl', 'rb') as f:
            #rank_dict = pickle.load(f)
        for line in f:
            # Ignore SubObjectPropertyOf
            if line.startswith('SubObjectPropertyOf'):
                line = line.strip()[20:-1]
                if line.startswith('ObjectPropertyChain'):
                    line_chain = line.strip()[20:-1]
                    line1 = line.split(")")
                    line10 = line1[0].split()
                    r1 = line10[0]
                    r2 = line10[1]
                    r3 = line1[1]
                    #if '>' not in r1:
                      #print(r1)
                    if r1 not in relations:
                        relations[r1] = len(relations)
                    if r2 not in relations:
                        relations[r2] = len(relations)
                    if r3 not in relations:
                        relations[r3] = len(relations)
                    data['nf_chain'].append((relations[r1],relations[r2],relations[r3]))
                else:
#                     print("Inside sub obj prop")
                    it = line.split(' ')
                    r1 = it[0]
                    r2 = it[1]
                    if r1 not in relations:
                        relations[r1] = len(relations)
                    if r2 not in relations:
                        relations[r2] = len(relations)
                    data['nf_inclusion'].append((relations[r1], relations[r2]))
            # Ignore SubClassOf()
            #line = line.strip()[11:-1]
            if not line:
                continue

            """
            if line.startswith('InstanceOf'):
                line = line.strip()[11:-1]
                it = line.split(' ')
                c = it[0]
                d = it[1]
                r = 'InstanceOf'
                if r not in relations:
                    relations[r] = len(relations)
                if c not in instances:
                    instances[c] = len(instances)
                if d not in classes:
                    classes[d] = len(classes)
                data['nf1_ins'].append((instances[c],relations[r],classes[d]))"""

            if line.startswith('InstanceOf'):
                line = line.strip()[11:-1]
                #print(line)
                it = line.split(' ')
                c = it[1]
                d = it[2]
                i = it[0]
                #c_rad=int(it[3])
                #d_rad=int(it[4])
                r = 'InstanceOf'
                if i not in instances:
                    instances[i] = len(instances)
                if r not in relations:
                    relations[r] = len(relations)
                if c not in classes:
                    classes[c] = len(classes)
                if d not in classes:
                    classes[d] = len(classes)
                data['nf1_ins'].append((instances[i],relations[r],classes[c],classes[d]))

            if line.startswith('SubClassOf'):
             line = line.strip()[11:-1]
             if line.startswith('ObjectIntersectionOf('):
                # C and D SubClassOf E
              if 'CL_' not in line:
                it = line.split(' ')
                c = it[0][21:]
                d = it[1][:-1]
                e = it[2]
                if '>' not in d:
                      print(d)
                if c not in classes:
                    classes[c] = len(classes)
                if d not in classes:
                    classes[d] = len(classes)
                if e not in classes:
                    classes[e] = len(classes)
                form = 'nf2'
                if e == 'owl:Nothing':
                    form = 'disjoint'
                data[form].append((classes[c], classes[d], classes[e]))

             elif line.startswith('ObjectSomeValuesFrom('):
                # R some C SubClassOf D
                it = line.split(' ')
                r = it[0][21:]
                c = it[1][:-1]
                d = it[2]
                #if '>' not in c:
                      #print(c)
                if c not in classes:
                    classes[c] = len(classes)
                if d not in classes:
                    classes[d] = len(classes)
                if r not in relations:
                    relations[r] = len(relations)
                data['nf4'].append((relations[r], classes[c], classes[d]))
             elif line.find('ObjectSomeValuesFrom') != -1:
                # C SubClassOf R some D
                it = line.split(' ')
                c = it[0]
                r = it[1][21:]
                d = it[2][:-1]
                if '>' not in d:
                      print(d)
                if c not in classes:
                    classes[c] = len(classes)
                if d not in classes:
                    classes[d] = len(classes)
                if r not in relations:
                    relations[r] = len(relations)
                data['nf3'].append((classes[c], relations[r], classes[d]))
             else:
                # C SubClassOf D
                #print(line)
                it = line.split(' ')
                #print(it)
                c = it[0]
                d = it[1]
                if '>' not in d:
                      print(line)
                      print(d)
                #c_rad= int(it[2])
                #d_rad= int(it[3])
                r = 'SubClassOf'
                if r not in relations:
                    relations[r] = len(relations)
                if c not in classes:
                    classes[c] = len(classes)
                if d not in classes:
                    classes[d] = len(classes)
                data['nf1'].append((classes[c],relations[r],classes[d]))

    # Check if TOP in classes and insert if it is not there
    if 'owl:Thing' not in classes:
        classes['owl:Thing'] = len(classes)
#changing by adding sub classes of train_data ids to prot_ids
    prot_id_cls=[]
    for val in classes:
              prot_id_cls.append(classes[val])
    prot_id_cls=np.array(prot_id_cls)

    # Add corrupted triples nf3
    n_classes = len(classes)
    data['nf3_neg'] = []
    for c, r, d in data['nf3']:
        x = np.random.choice(prot_id_cls)
        while x == c:
            x = np.random.choice(prot_id_cls)

        y = np.random.choice(prot_id_cls)
        while y == d:
             y = np.random.choice(prot_id_cls)
        data['nf3_neg'].append((c, r,x))
        data['nf3_neg'].append((y, r, d))




    data['nf1_neg'] = []

    for c, r, d in data['nf1']:
        x = np.random.choice(prot_id_cls)
        while x == c:
            x = np.random.choice(prot_id_cls)

        y = np.random.choice(prot_id_cls)
        while y == d:
             y = np.random.choice(prot_id_cls)
        data['nf1_neg'].append((c, r,x))
        data['nf1_neg'].append((y, r, d))


    data['radius'] = []
    for val in classes:
        rnk=rank_dict[val]
        data['radius'].append((classes[val],rnk))
    """
    data['radius'] = []
    for val in classes:
        data['radius'].append(classes[val])"""

    data['nf1'] = np.array(data['nf1'])
    data['nf1_ins'] = np.array(data['nf1_ins'])
    data['nf2'] = np.array(data['nf2'])
    data['nf3'] = np.array(data['nf3'])
    data['nf4'] = np.array(data['nf4'])
    data['disjoint'] = np.array(data['disjoint'])
    data['top'] = np.array([classes['owl:Thing'],])
    data['nf1_neg'] = np.array(data['nf1_neg'])
    data['nf3_neg'] = np.array(data['nf3_neg'])
    data['nf_inclusion'] = np.array(data['nf_inclusion'])
    data['nf_chain'] = np.array(data['nf_chain'])
    data['radius'] = np.array(data['radius'])

    for key, val in data.items():
        index = np.arange(len(data[key]))
        np.random.seed(seed=100)
        np.random.shuffle(index)
        data[key] = val[index]

    return data, classes, relations, instances

#gdata_file="HPO_data_25Feb/HPO_metadata_finalnf1_25Feb2_3.owl"
#train_data_model, classes, relations,instances = load_data(gdata_file)

In [ ]:
class ELModel(tf.keras.Model):

    def __init__(self, nb_classes, nb_relations, nb_instances,embedding_size, batch_size, margin, reg_norm=1):
        super(ELModel, self).__init__()
        self.nb_classes = nb_classes
        self.nb_relations = nb_relations
        self.nb_instances= nb_instances
        self.margin = margin
        self.reg_norm = reg_norm
        self.batch_size = batch_size
        self.inf = 5.0 # For top radius
        #initialization of class weights and radius
        csample_weights = np.random.uniform(low=-1, high=1, size=(nb_classes, embedding_size))
        radius_wts = np.random.uniform(low=0, high=1, size=(nb_classes,))
        cls_weights = np.column_stack((csample_weights,radius_wts))
        cls_weights = cls_weights / np.linalg.norm(
            cls_weights, axis=1).reshape(-1, 1)
        #rel bias
        b1rad_wts = np.random.uniform(low=-1, high=1, size=(nb_relations,embedding_size))
        b2rad_wts = np.random.uniform(low=-1, high=1, size=(nb_relations,embedding_size))
        #rel bias
        #rel_weights = np.column_stack((csample_weights,radius_wts))
        rel_weights = np.random.uniform(low=-1, high=1, size=(nb_relations, embedding_size))
        #rel_weights = np.column_stack((rel_weights,b1rad_wts,b2rad_wts)) ######
        rel_weights= rel_weights / np.linalg.norm(
            rel_weights, axis=1).reshape(-1, 1)
        #embedding weights
        mat_weights = np.random.uniform(low=-1, high=1, size=(embedding_size, embedding_size))
        #mat_weights = np.ones(shape=(embedding_size, embedding_size))
        #mat_weights=mat_weights.reshape(-1,1)
        mat_weights = mat_weights / np.linalg.norm(
            mat_weights, axis=1).reshape(-1, 1)
        ###instances
        isample_weights = np.random.uniform(low=-1, high=1, size=(nb_instances, embedding_size))
        ins_weights = isample_weights / np.linalg.norm(
            isample_weights, axis=1).reshape(-1, 1)
        #embedding weights
        self.cls_embeddings = tf.keras.layers.Embedding(
            nb_classes,
            embedding_size + 1,
            input_length=1,
            weights=[cls_weights,])
        self.rel_embeddings = tf.keras.layers.Embedding(
            nb_relations,
            embedding_size ,######
            input_length=1,
            weights=[rel_weights,])
        self.mat_embeddings = tf.keras.layers.Embedding(
            embedding_size,
            embedding_size,
            input_length=1,
            weights=[mat_weights,])#########
        ###instance embedding
        self.ins_embeddings = tf.keras.layers.Embedding(
            nb_instances,
            embedding_size,
            input_length=1,
            weights=[ins_weights,])


    def call(self, input):
        """Run the model."""
        nf1, nf1_ins,nf2, nf3, nf4, dis, top, nf1_neg,nf3_neg,nf_inclusion,nf_chain,radius, nf_matrix = input
        loss1 = self.nf1_loss(nf1)
        loss1a = self.nf1_ins_loss(nf1_ins,nf_matrix)
        loss2 = self.nf2_loss(nf2)
        loss3 = self.nf3_loss(nf3)
        loss4 = self.nf4_loss(nf4)
        loss_dis = self.dis_loss(dis)
        loss_top = self.top_loss(top)
        #loss_nf1_neg = self.nf1_neg_loss(nf1_neg)
        loss_nf3_neg = self.nf3_neg_loss(nf3_neg)
        loss5 = self.inclusion_loss(nf_inclusion)
        loss6 = self.chain_loss(nf_chain)
        loss7 = self.radius_loss(radius)
        loss = loss1 + loss1a+loss2 + loss3 + loss4 + loss_dis + loss_top + loss_nf3_neg + loss5 + loss6 - loss7
        return loss


    def reg(self, x):
        res = tf.abs(tf.norm(x, axis=1) - self.reg_norm)
        res = tf.reshape(res, [-1, 1])
        return res


    def nf1_loss(self, input):
        c = input[:, 0]
        r = input[:,1]
        d = input[:, 2]
        c = self.cls_embeddings(c)
        d = self.cls_embeddings(d)
        r = self.rel_embeddings(r)
        #c_deg=tf.math.sqrt(tf.math.sqrt(tf.cast(input[:,3], tf.float32)))
        #d_deg=tf.math.sqrt(tf.math.sqrt(tf.cast(input[:,4], tf.float32)))



        rc = tf.math.maximum(0.0,c[:, -1])
        rd = tf.math.maximum(0.0,d[:, -1])
        x1 = c[:, 0:-1]
        x2 = d[:, 0:-1]
        x3 = x1 + r
        euc = tf.norm(x3 - x2, axis=1)
        dst = tf.reshape(tf.nn.relu(euc + rc - rd - (-0.4)), [-1, 1])
        return dst + self.reg(x1) + self.reg(x2)
    """
    def nf1_loss(self, input):
        c = input[:, 0]
        d = input[:, 2]
        c = self.cls_embeddings(c)
        d = self.cls_embeddings(d)

        rc = tf.reshape(tf.math.abs(c[:, -1]), [-1, 1])
        rd = tf.reshape(tf.math.abs(d[:, -1]), [-1, 1])
        x1 = c[:, 0:-1]
        x2 = d[:, 0:-1]
        # x1 = x1 / tf.reshape(tf.norm(x1, axis=1), [-1, 1])
        # x2 = x2 / tf.reshape(tf.norm(x2, axis=1), [-1, 1])
        euc = tf.reshape(tf.norm(x1 - x2, axis=1), [-1, 1])
        dst = tf.nn.relu(euc + rc - rd - self.margin)
        return dst + self.reg(x1) + self.reg(x2)"""



    """
    def nf1_ins_loss(self, input):
        c = input[:, 0]
        d = input[:, 2]  #i
        c = self.ins_embeddings(c)
        d = self.cls_embeddings(d)

        rd = tf.math.maximum(0.0,d[:, -1])
        x1 = c[:, 0:]
        x2 = d[:, 0:-1]
        euc = tf.norm(x1 - x2, axis=1)
        dst = tf.reshape(tf.nn.relu(euc -rd - self.margin), [-1, 1])
        return dst + self.reg(x1) +self.reg(x2)"""
    """
    def nf1_ins_loss(self, input,input2):
        c = input[:, 2] #c
        d = input[:, 3]  #d
        i=input[:,0] #i

        #c_deg=tf.math.sqrt(tf.math.sqrt(tf.cast(input[:,4], tf.float32)))
        #d_deg=tf.math.sqrt(tf.math.sqrt(tf.cast(input[:,5], tf.float32)))
        #c_deg=tf.math.sqrt(10.00-tf.cast(input[:,4], tf.float32))
        #d_deg=tf.math.sqrt(10.00-tf.cast(input[:,5], tf.float32))
        #deg=tf.reshape()
        #print(c_deg.shape,i.shape)
        m=input2[:,0]
        m=self.mat_embeddings(m)
        c = self.cls_embeddings(c)
        d = self.cls_embeddings(d)
        i = self.ins_embeddings(i)

        #rd = tf.math.maximum(0.0,d[:, -1])
        #rc = tf.math.maximum(0.0,c[:, -1])
        rda = tf.math.maximum(0.0,d[:, -1])
        rca = tf.math.maximum(0.0,c[:, -1])
        #rca=tf.cast(input[:,4], tf.float32)
        #rda=tf.cast(input[:,5], tf.float32)

        x1 = c[:, 0:-1]
        x2 = d[:, 0:-1]
        x3= i[:,0:]
        euc1 = tf.norm(x1 - x2, axis=1)
        euc2 = tf.norm(x1 - x3, axis=1)
        euc3=tf.norm(x2 - x3, axis=1)
        #dst1 = tf.reshape(tf.nn.relu(euc1-rca -rda - self.margin), [-1, 1])
        #dst1 = tf.reshape(tf.nn.relu(euc1+rca -rda - self.margin), [-1, 1])
        dst2 = tf.reshape(tf.nn.relu(euc2-rca - self.margin), [-1, 1])
        #dst3 = tf.reshape(tf.nn.relu(euc3-rda - self.margin), [-1, 1])
        return dst2 + self.reg(x1)+self.reg(x2)+self.reg(x3)"""


    def nf1_ins_loss(self, input,input2):
         c = input[:, 2] #c
         d = input[:, 3]  #d
         i=input[:,0]
         m=input2[:,0]

         c = self.cls_embeddings(c)
         d = self.cls_embeddings(d)
         i = self.ins_embeddings(i)
         m=self.mat_embeddings(m)

         rd = tf.math.maximum(0.0,d[:, -1])
         rc = tf.math.maximum(0.0,c[:, -1])
         rda = tf.math.maximum(0.0,d[:, -1])
         rca = tf.math.maximum(0.0,c[:, -1])

         #x1 = c[:, 0:-1]
         x1_ = c[:, 0:-1]
         x2 = d[:, 0:-1]
         #x3_= i[:,0:]
         x3= i[:,0:]
         ##x3=tf.matmul(m,tf.transpose(x3_))
         x1=tf.matmul(m,tf.transpose(x1_))
         #x3=tf.transpose(x3)
         x1=tf.transpose(x1)
         euc1 = tf.norm(x1 - x2, axis=1)
         euc2 = tf.norm(x1 - x3, axis=1)
         dst2 = tf.reshape(tf.nn.relu(euc2-rca - self.margin), [-1, 1])
         return dst2 + self.reg(x1)+self.reg(x2)+self.reg(x3)








    def nf1_neg_loss(self, input):
        # C subClassOf R some D
        c = input[:, 0]
        r = input[:, 1]
        d = input[:, 2]
        c = self.cls_embeddings(c)
        d = self.cls_embeddings(d)
        r = self.rel_embeddings(r)
        x1 = c[:, 0:-1]
        x2 = d[:, 0:-1]
        #x1_r=c[:,-1]
        #x2_r=d[:,-1]

        x3 = x1 + r
        #diff_rad=x2_r-x1_r
        #euc = tf.norm(x1 - x2, axis=1)

        rc = tf.math.maximum(0.0,c[:, -1])
        rd = tf.math.maximum(0.0,d[:, -1])
        conda=tf.where(tf.less(rc,rd),[1.0],[0.0])
        #conda2=tf.broadcast_to(conda,[256,])
        euc = tf.norm(x2 - x1, axis=1)
        dst = tf.reshape(conda*(-(euc - rd + rc) + self.margin), [-1, 1])
        #dst = tf.reshape((-(euc - rd + rc) + self.margin), [-1, 1])
        #def f1(): return -(euc - rd + rc) + self.margin)\
        #dst1=tf.cond(rd>rc,f1,f1)
        #def f2(): return tf.math.maximum(0.0,0.0)
        #dst2=tf.cond(rd>rc,lambda: tf.reshape((-(euc - rd + rc) + self.margin), [-1, 1]),\
              #   lambda: tf.reshape((-(euc - rd + rc) + self.margin), [-1, 1])    )
        #dst2=tf.reshape(dst1,[-1,1])

        return tf.nn.relu(dst) + self.reg(x1) + self.reg(x2)

    def nf2_loss(self, input):
        c = input[:, 0]
        d = input[:, 1]
        e = input[:, 2]
        c = self.cls_embeddings(c)
        d = self.cls_embeddings(d)
        e = self.cls_embeddings(e)
        rc = tf.reshape(tf.math.maximum(0.0,c[:, -1]), [-1, 1])
        rd = tf.reshape(tf.math.maximum(0.0,d[:, -1]), [-1, 1])
        re = tf.reshape(tf.math.maximum(0.0,e[:, -1]), [-1, 1])
        sr = rc + rd
        x1 = c[:, 0:-1]
        x2 = d[:, 0:-1]
        x3 = e[:, 0:-1]

        x = x2 - x1
        dst = tf.reshape(tf.norm(x, axis=1), [-1, 1])
        dst2 = tf.reshape(tf.norm(x3 - x1, axis=1), [-1, 1])
        dst3 = tf.reshape(tf.norm(x3 - x2, axis=1), [-1, 1])
        rdst = tf.nn.relu(tf.math.minimum(rc, rd) - re - self.margin)
        dst_loss = (tf.nn.relu(dst - sr - self.margin)
                    + tf.nn.relu(dst2 - rc - self.margin)
                    + tf.nn.relu(dst3 - rd - self.margin)
                    + rdst)
        return dst_loss + self.reg(x1) + self.reg(x2) + self.reg(x3)

    def nf3_loss(self, input):
        # C subClassOf R some D
        c = input[:, 0]
        r = input[:, 1]
        d = input[:, 2]
        c = self.cls_embeddings(c)
        d = self.cls_embeddings(d)
        r = self.rel_embeddings(r)
        x1 = c[:, 0:-1]
        x2 = d[:, 0:-1]
        x3 = x1 + r

        rc = tf.math.maximum(0.0,c[:, -1])
        rd = tf.math.maximum(0.0,d[:, -1])
        euc = tf.norm(x3 - x2, axis=1)
        dst = tf.reshape(tf.nn.relu(euc + rc - rd - self.margin), [-1, 1])

        return dst + self.reg(x1) + self.reg(x2)

    def nf3_neg_loss(self, input):
        # C subClassOf R some D
        c = input[:, 0]
        r = input[:, 1]
        d = input[:, 2]
        c = self.cls_embeddings(c)
        d = self.cls_embeddings(d)
        r = self.rel_embeddings(r)
        x1 = c[:, 0:-1]
        x2 = d[:, 0:-1]

        x3 = x1 + r

        rc = tf.math.maximum(0.0,c[:, -1])
        rd = tf.math.maximum(0.0,d[:, -1])
        euc = tf.norm(x3 - x2, axis=1)
        dst = tf.reshape((-(euc - rc - rd) + self.margin), [-1, 1])

        return dst + self.reg(x1) + self.reg(x2)

    def nf4_loss(self, input):
        # R some C subClassOf D
        r = input[:, 0]
        c = input[:, 1]
        d = input[:, 2]
        c = self.cls_embeddings(c)
        d = self.cls_embeddings(d)
        r = self.rel_embeddings(r)
        rc = tf.reshape(tf.math.maximum(0.0,c[:, -1]), [-1, 1])
        rd = tf.reshape(tf.math.maximum(0.0,d[:, -1]), [-1, 1])
        sr = rc + rd
        x1 = c[:, 0:-1]
        x2 = d[:, 0:-1]

        # c - r should intersect with d
        x3 = x1 - r
        dst = tf.reshape(tf.norm(x3 - x2, axis=1), [-1, 1])
        dst_loss = tf.nn.relu(dst - sr - self.margin)
        return dst_loss + self.reg(x1) + self.reg(x2)


    def dis_loss(self, input):
        c = input[:, 0]
        d = input[:, 1]
        c = self.cls_embeddings(c)
        d = self.cls_embeddings(d)
        rc = tf.reshape(tf.math.maximum(0.0,c[:, -1]), [-1, 1])
        rd = tf.reshape(tf.math.maximum(0.0,d[:, -1]), [-1, 1])
        sr = rc + rd
        x1 = c[:, 0:-1]
        x2 = d[:, 0:-1]

        dst = tf.reshape(tf.norm(x2 - x1, axis=1), [-1, 1])
        return tf.nn.relu(sr - dst + self.margin) + self.reg(x1) + self.reg(x2)


    def top_loss(self, input):
        d = input[:, 0]
        d = self.cls_embeddings(d)
        rd = tf.reshape(tf.math.maximum(0.0,d[:, -1]), [-1, 1])
        return tf.math.abs(rd - self.inf)

    def inclusion_loss(self,input):
        r1 = input[:, 0]
        r2 = input[:, 1]
        r1 = self.rel_embeddings(r1)
        r2 = self.rel_embeddings(r2)
        #print("r2 type------>",type(r2))

        euc = tf.norm(r2 - r1, axis=1)

        normalize_a = tf.nn.l2_normalize(r1,0)
        normalize_b = tf.nn.l2_normalize(r2,0)
        direction=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
        dir_loss = tf.abs(1 - direction)
        dir_loss = tf.reshape(dir_loss, [-1, 1])
        dst = tf.reshape(tf.nn.relu(euc - self.margin), [-1, 1])

        return dst + self.reg(r1) + self.reg(r2) + dir_loss

    def chain_loss(self,input):
        r1 = input[:, 0]
        r2 = input[:, 1]
        r3 = input[:, 2]
        c = self.rel_embeddings(r1)
        d = self.rel_embeddings(r2)
        e = self.rel_embeddings(r3)
        s = e - c

        dst = tf.reshape(tf.norm(c - d, axis=1), [-1, 1])
        dst2 = tf.reshape(tf.norm(e - c, axis=1), [-1, 1])
        dst3 = tf.reshape(tf.norm(e - d, axis=1), [-1, 1])
        dst_dir = tf.reshape(tf.norm(s - d, axis=1), [-1, 1])
        dst_loss = (tf.nn.relu(dst_dir - self.margin))

        res_vec = c + d
        normalize_a = tf.nn.l2_normalize(res_vec,0)
        normalize_b = tf.nn.l2_normalize(e,0)
        direction=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
        dir_loss = tf.abs(1 - direction)
        dir_loss = tf.reshape(dir_loss, [-1, 1])
        return dst_loss + self.reg(c) + self.reg(d) + self.reg(e) + dir_loss
    """
    def radius_loss(self, input):
        d = input[:, 0]
        d = self.cls_embeddings(d)
        rd = tf.reshape(d[:, -1], [-1, 1])
        return tf.math.minimum(0.0,rd)"""

    def radius_loss(self,input):
        d=input[:,0]
        d_level=input[:,1]
        d_level=tf.cast(d_level, tf.float32)
        d=self.cls_embeddings(d)
        rd = tf.reshape(d[:, -1], [-1, 1])
        conda1=tf.where(tf.equal(0.0,d_level),1.0,0.0)
        conda2=tf.where(tf.equal(0.0,d_level),0.0,1.0)
        dst1=conda1*tf.math.minimum(0.0,rd)+(-0.2)
        dst2=conda2*tf.math.minimum(0.0,tf.math.sqrt(tf.cast(d_level, tf.float32))*rd)
        return dst1+dst2
        #conda2=tf.broadcast_to(conda[0],[256,])


# In[10]:


class Generator(object):

    def __init__(self, data, batch_size, steps=100):
        self.data = data
        self.batch_size = batch_size
        self.steps = steps
        self.start = 0

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()

    def reset(self):
        self.start = 0

    def next(self):
        if self.start < self.steps:
            nf1_index = np.random.choice(
                self.data['nf1'].shape[0], self.batch_size)
            nf1_ins_index = np.random.choice(
                self.data['nf1_ins'].shape[0], self.batch_size)
            nf2_index = np.random.choice(
                self.data['nf2'].shape[0], self.batch_size)
            nf3_index = np.random.choice(
                self.data['nf3'].shape[0], self.batch_size)
            nf4_index = np.random.choice(
                self.data['nf4'].shape[0], self.batch_size)
            dis_index = np.random.choice(
                self.data['disjoint'].shape[0], self.batch_size)
            top_index = np.random.choice(
                self.data['top'].shape[0], self.batch_size)
            nf1_neg_index = np.random.choice(
               self.data['nf1_neg'].shape[0], self.batch_size)
            nf3_neg_index = np.random.choice(
                self.data['nf3_neg'].shape[0], self.batch_size)
            nf_inclusion_index = np.random.choice(self.data['nf_inclusion'].shape[0],self.batch_size)
            #nf_inclusion_index = np.random.choice(self.data['nf_inclusion'].shape[0],101)
            nf_chain_index = np.random.choice(self.data['nf_chain'].shape[0],self.batch_size)
            radius_index = np.random.choice(
                self.data['radius'].shape[0], self.batch_size)
            nf1 = self.data['nf1'][nf1_index]
            nf1_ins = self.data['nf1_ins'][nf1_ins_index]
            nf2 = self.data['nf2'][nf2_index]
            nf3 = self.data['nf3'][nf3_index]
            nf4 = self.data['nf4'][nf4_index]
            dis = self.data['disjoint'][dis_index]
            top = self.data['top'][top_index]
            nf1_neg = self.data['nf1_neg'][nf1_neg_index]
            nf3_neg = self.data['nf3_neg'][nf3_neg_index]
            nf_inclusion = self.data['nf_inclusion'][nf_inclusion_index]
            nf_chain = self.data['nf_chain'][nf_chain_index]
            nf_matrix=np.arange(0,100,1)
            radius = self.data['radius'][radius_index]
            labels = np.zeros((self.batch_size, 1), dtype=np.float32)
            self.start += 1
            return ([nf1,nf1_ins, nf2, nf3, nf4, dis, top, nf1_neg,nf3_neg,nf_inclusion,nf_chain,radius,nf_matrix], labels)
        else:
            self.reset()


# In[11]:


class MyModelCheckpoint(ModelCheckpoint):

    def __init__(self, *args, **kwargs):
        super(ModelCheckpoint, self).__init__()
        self.out_classes_file = kwargs.pop('out_classes_file')
        self.out_relations_file = kwargs.pop('out_relations_file')
        self.out_instances_file = kwargs.pop('out_instances_file')
        self.out_matrix_file = kwargs.pop('out_matrix_file')########
        self.monitor = kwargs.pop('monitor')
        self.cls_list = kwargs.pop('cls_list')
        self.rel_list = kwargs.pop('rel_list')
        self.load_weights_on_restart = False
        ##
        self.save_freq='epoch'
        self.ins_list = kwargs.pop('ins_list')###
        self.valid_data = kwargs.pop('valid_data')
        self.proteins_cls = kwargs.pop('proteins_cls')
        self.proteins_ins = kwargs.pop('proteins_ins')
        self.prot_index_c= list(self.proteins_cls.values())
        self.prot_dict_c = {v: k for k, v in enumerate(self.prot_index_c)}
        self.prot_index_i = list(self.proteins_ins.values())
        self.prot_dict_i = {v: k for k, v in enumerate(self.prot_index_i)}

        self.best_rank = 100000


    def on_epoch_end(self, epoch, logs=None):
        # Save embeddings every 10 epochs
        current_loss = logs.get(self.monitor)
        if math.isnan(current_loss):
            print('NAN loss, stopping training')
            self.model.stop_training = True
            return
        el_model = self.model.layers[-1]
        cls_embeddings = el_model.cls_embeddings.get_weights()[0]
        rel_embeddings = el_model.rel_embeddings.get_weights()[0]
        ins_embeddings = el_model.ins_embeddings.get_weights()[0]
        mat_embeddings = el_model.mat_embeddings.get_weights()[0]

        #prot_embeds = cls_embeddings[self.prot_index]
        #prot_rs = prot_embeds[:, -1].reshape(-1, 1)
        #prot_embeds = prot_embeds[:, :-1]
        mean_rank = 0
        n = len(self.valid_data)

        for c, r, d in self.valid_data:
            #if r==relations['SubClassOf']:
              #print('yes')
              prot_embeds = cls_embeddings[self.prot_index_c]
              prot_rs = prot_embeds[:, -1].reshape(-1, 1)
              prot_embeds = prot_embeds[:, :-1]
              c, r, d = self.prot_dict_c[c], r, self.prot_dict_c[d]
              ec = prot_embeds[c, :]
              rc = prot_rs[c, :]
              er = rel_embeddings[r, :]
              ec += er
              #ec=tf.matmul(m,tf.transpose(d[:, 0:-2]))
              dst = np.linalg.norm(prot_embeds - ec.reshape(1, -1), axis=1)
              dst = dst.reshape(-1, 1)
              res = np.maximum(0, dst - rc - prot_rs - el_model.margin)
              res = res.flatten()
              index = rankdata(res, method='average')
              rank = index[d]
              mean_rank += rank

        mean_rank /= n
        # fmean_rank /= n
        print(f'\n Validation {epoch + 1} {mean_rank}\n')
        if mean_rank < self.best_rank:
            self.best_rank = mean_rank
            print(f'\n Saving embeddings {epoch + 1} {mean_rank}\n')
            cls_file = self.out_classes_file
            rel_file =self.out_relations_file
            ins_file = self.out_instances_file
            out_matrix= self.out_matrix_file
            # Save embeddings of every thousand epochs
            # if (epoch + 1) % 1000 == 0:
            # cls_file = f'{cls_file}_{epoch + 1}.pkl'
            # rel_file = f'{rel_file}_{epoch + 1}.pkl'

            df = pd.DataFrame(
                {'classes': self.cls_list, 'embeddings': list(cls_embeddings)})
            df.to_pickle(cls_file)

            df = pd.DataFrame(
                {'relations': self.rel_list, 'embeddings': list(rel_embeddings)})
            df.to_pickle(rel_file)
            #mat_embeddings_nmpy=mat_embeddings.numpy()

            df = pd.DataFrame(
                {'instances': self.ins_list, 'embeddings': list(ins_embeddings)})
            df.to_pickle(ins_file)


            mat_embeddings_nmpy=np.array(mat_embeddings)
            #out_matrix=f'{rel_file}_{epoch + 1}.npy'
            #os.remove('Matrix.npy')
            out_matrix=np.save('Matrix.npy',mat_embeddings_nmpy)



# In[12]:


def build_model(device,train_data,classes,instances,relations,valid_data):
    proteins_cls = {}#substitute for classes with subclass case
    #for val in all_subcls:
    for val in classes:
        proteins_cls[val] = classes[val]
    proteins_ins = {}#substitute for classes with instances case
    #for val in all_subcls:
    for val in instances:
        proteins_ins[val] = instances[val]
    nb_classes = len(classes)
    nb_relations = len(relations)
    nb_instances= len(instances)###
    print("no. classes:",nb_classes)
    print("no. relations:",nb_relations)
    print("no. instances:",nb_instances)###
    nb_train_data = 0
    for key, val in train_data.items():
        nb_train_data = max(len(val), nb_train_data)
    train_steps = int(math.ceil(nb_train_data / (1.0 * batch_size)))
    train_generator = Generator(train_data, batch_size, steps=train_steps)

    cls_dict = {v: k for k, v in classes.items()}
    rel_dict = {v: k for k, v in relations.items()}
    ins_dict = {v: k for k, v in instances.items()}####

    cls_list = []
    rel_list = []
    ins_list=[]
    for i in range(nb_classes):
        cls_list.append(cls_dict[i])
    for i in range(nb_relations):
        rel_list.append(rel_dict[i])
    for i in range(nb_instances):##
        ins_list.append(ins_dict[i])##

    with tf.device('/' + device):
        nf1 = Input(shape=(3,), dtype=np.int32)
        nf1_ins = Input(shape=(4,), dtype=np.int32)
        nf2 = Input(shape=(3,), dtype=np.int32)
        nf3 = Input(shape=(3,), dtype=np.int32)
        nf4 = Input(shape=(3,), dtype=np.int32)
        dis = Input(shape=(3,), dtype=np.int32)
        top = Input(shape=(1,), dtype=np.int32)
        nf1_neg = Input(shape=(3,), dtype=np.int32)
        nf3_neg = Input(shape=(3,), dtype=np.int32)
        nf_inclusion = Input(shape=(2,), dtype=np.int32)
        nf_matrix = Input(shape=(1,), dtype=np.int32)
        nf_chain = Input(shape=(3,), dtype=np.int32)
        radius = Input(shape=(2,), dtype=np.int32)
        el_model = ELModel(nb_classes, nb_relations, nb_instances,embedding_size, batch_size, margin, reg_norm)
        out = el_model([nf1,nf1_ins, nf2, nf3, nf4,  dis,top,nf1_neg, nf3_neg, nf_inclusion, nf_chain,radius,nf_matrix])
        model = tf.keras.Model(inputs=[nf1,nf1_ins, nf2, nf3, nf4,dis,top,nf1_neg, nf3_neg,nf_inclusion,nf_chain,radius,nf_matrix], outputs=out)
        optimizer = optimizers.Adam(lr=learning_rate)
        model.compile(optimizer=optimizer, loss='mse')
    # TOP Embedding
        top = classes.get('owl:Thing', None)
        checkpointer = MyModelCheckpoint(
            out_classes_file=out_classes_file,
            out_relations_file=out_relations_file,
            out_instances_file=out_instances_file,
            out_matrix_file=out_matrix_file,
            cls_list=cls_list,
            rel_list=rel_list,
            ins_list=ins_list,
            #out_matrix=out_matrix,
            valid_data=valid_data[0:100],
            proteins_cls=proteins_cls,
            proteins_ins=proteins_ins,
            monitor='loss')

        logger = CSVLogger(loss_history_file)

        # Save initial embeddings
        cls_embeddings = el_model.cls_embeddings.get_weights()[0]
        rel_embeddings = el_model.rel_embeddings.get_weights()[0]
        ins_embeddings = el_model.ins_embeddings.get_weights()[0]
        mat_embeddings = el_model.mat_embeddings.get_weights()[0]

        cls_file = out_classes_file
        rel_file = out_relations_file
        ins_file= out_instances_file
        out_matrix= out_matrix_file

        df = pd.DataFrame(
            {'classes': cls_list, 'embeddings': list(cls_embeddings)})
        df.to_pickle(cls_file)
        print('SHAPES_CRITICAL_REL',len(rel_list),len(list(rel_embeddings)),rel_embeddings.shape)
        df = pd.DataFrame(
            {'relations': rel_list, 'embeddings': list(rel_embeddings)})
        df.to_pickle(rel_file)

        print('SHAPES_CRITICAL',len(ins_list),len(list(ins_embeddings)),ins_embeddings.shape)
        df = pd.DataFrame(
            {'instances': ins_list, 'embeddings': list(ins_embeddings)})
        df.to_pickle(ins_file)
        #mat_embeddings_nmpy=mat_embeddings.numpy()
        mat_embeddings_nmpy=np.array(mat_embeddings)
        #os.remove('Matrix.npy')
        out_matrix=np.save('Matrix.npy',mat_embeddings_nmpy)

        model.fit_generator(
            train_generator,
            steps_per_epoch=train_steps,
            epochs=epochs,
            workers=12,
            callbacks=[logger, checkpointer])


# In[15]:


gdata_file="HPO_all_Mar1/HPO_metadata_finalnf1_26Feb_all_.owl"
train_data_model, classes, relations,instances = load_data(gdata_file)


# In[16]:


valid_data_file="HPO_all_Mar1/HPO_valid.txt"
valid_data_model = load_valid_data(valid_data_file, classes, instances,relations)


# In[17]:


margins = [-0.1]
embed_dims = [100]
batch_size =  256
device='gpu:0'
reg_norm=1
learning_rate=3e-4
epochs=1000
for d in embed_dims:
    embedding_size = d

    print("***************Embedding Dim:",embedding_size,'****************')
    for m in margins:
        margin = m
        print("**************Margin Loss:",margin,"***************")
        out_classes_file = f'HPO_results/EmEL_dir/hpo_{embedding_size}_{margin}_{epochs}_clshpo_data_p2_Mall2.pkl'
        out_relations_file = f'HPO_results/EmEL_dir/hpo_{embedding_size}_{margin}_{epochs}_relhpo_data_p2_Mall2.pkl'
        out_instances_file = f'HPO_results/EmEL_dir/hpo_{embedding_size}_{margin}_{epochs}_ins_hpo_data_p2_Mall2.pkl'
        #loss_history_file= f'GO_results/EmEL_dir/GO_lossHis_{embedding_size}_{margin}_{epochs}.csv'
        out_matrix_file=f'HPO_results/EmEL_dir/GO_lossHis_{embedding_size}_{margin}_{epochs}.npy'
        loss_history_file= f'HPO_results/EmEL_dir/GO_lossHis_{embedding_size}_{margin}_{epochs}.csv'
        #out_instanceout_matrixs_file = f'YAGO_results/EmEL_dir/YAGO_{embedding_size}_{margin}_{epochs}_ins.csv'
        build_model(device,train_data_model,classes,instances,relations,valid_data_model)

***************Embedding Dim: 100 ****************
**************Margin Loss: -0.1 ***************
no. classes: 30520
no. relations: 326
no. instances: 12115


SHAPES_CRITICAL_REL 326 326 (326, 100)
SHAPES_CRITICAL 12115 12115 (12115, 100)
Epoch 1/1000


<ipython-input-3-00f680173433>:686: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


352/352 [==============================] - ETA: 0s - loss: 315.9825
 Validation 1 4772.6


 Saving embeddings 1 4772.6

352/352 [==============================] - 34s 81ms/step - loss: 315.9825
Epoch 2/1000
352/352 [==============================] - ETA: 0s - loss: 211.6335
 Validation 2 3529.19


 Saving embeddings 2 3529.19

352/352 [==============================] - 28s 80ms/step - loss: 211.6335
Epoch 3/1000
352/352 [==============================] - ETA: 0s - loss: 151.1737
 Validation 3 2828.51


 Saving embeddings 3 2828.51

352/352 [==============================] - 28s 79ms/step - loss: 151.1737
Epoch 4/1000
352/352 [==============================] - ETA: 0s - loss: 118.0826
 Validation 4 2479.895


 Saving embeddings 4 2479.895

352/352 [==============================] - 27s 78ms/step - loss: 118.0826
Epoch 5/1000
352/352 [==============================] - ETA: 0s - loss: 102.4377
 Validation 5 2227.99


 Saving embeddings 5 2227.99

352/352 [==============================] -

KeyboardInterrupt: 